* Sterne, Theodore E. Some Remarks on Confidence or Fiducial Limits. Biometrika
Vol. 41, No. 1/2 (Jun., 1954), pp. 275-278 (4 pages) https://www.jstor.org/stable/2333026

In [1]:
using Distributions
using StatsPlots
default(fmt=:png, titlefontsize=10)

In [18]:
x ⪅ y = x < y || x ≈ y

# Naive implementation is terrible slow.
function pvalue_stern_naive(dist::DiscreteUnivariateDistribution, x; xmax = 10^6)
    Px = pdf(dist, x)
    Px == 0 && return Px
    m = mode(dist)
    (x == m || Px ≈ pdf(dist, m)) && return 1.0
    ymin, maxdist = minimum(dist), maximum(dist)
    ymax = maxdist == Inf ? xmax : maxdist
    sum(pdf(dist, y) for y in ymin:ymax if 0 < pdf(dist, y) ⪅ Px; init = 0.0)
end

### efficient implementation

_pdf_le(x, (dist, y)) =  pdf(dist, x) ⪅ y

function _search_boundary(f, x0, Δx, param)
    x = x0
    if f(x, param)
        while f(x - Δx, param) x -= Δx end
    else
        x += Δx
        while !f(x, param) x += Δx end
    end
    x
end

function pvalue_stern(dist::DiscreteUnivariateDistribution, x)
    Px = pdf(dist, x)
    Px == 0 && return Px
    m = mode(dist)
    (x == m || Px ≈ pdf(dist, m)) && return 1.0
    if x < m
        y = _search_boundary(_pdf_le, 2m - x, 1, (dist, Px))
        cdf(dist, x) + ccdf(dist, y-1)
    else # x > m
        y = _search_boundary(_pdf_le, 2m - x, -1, (dist, Px))
        cdf(dist, y) + ccdf(dist, x-1)
    end
end

pvalue_stern (generic function with 1 method)

In [19]:
dist = Binomial(10, 0.3)
a = pvalue_stern.(dist, -1:11)
b = pvalue_stern_naive.(dist, -1:11)
@show a ≈ b
a

a ≈ b = true


13-element Vector{Float64}:
 0.0
 0.038839603299999984
 0.2995766784999999
 0.7331720679999996
 1.0
 0.4996976274999998
 0.17851585749999987
 0.07559651229999995
 0.010592078399999996
 0.0015903864000000002
 0.00014368589999999999
 5.904899999999991e-6
 0.0

In [24]:
dist = Binomial(100000, 0.5)
ran = 49500:50500
@time a = pvalue_stern.(dist, ran)
@time b = pvalue_stern_naive.(dist, ran)
@show a ≈ b
a

  0.001839 seconds (4 allocations: 8.094 KiB)
 10.855532 seconds (4 allocations: 8.094 KiB)
a ≈ b = true


1001-element Vector{Float64}:
 0.0015823598788515954
 0.0016170462998192799
 0.0016524313261302747
 0.0016885275841076281
 0.0017253478963870853
 0.001762905284304741
 0.0018012129703004164
 0.0018402843803366896
 0.0018801331463333246
 0.0019207731086169947
 0.0019622183183860276
 0.002004483040190035
 0.0020475817544242164
 ⋮
 0.002004483040190035
 0.0019622183183860276
 0.0019207731086169947
 0.0018801331463333246
 0.0018402843803366896
 0.0018012129703004164
 0.001762905284304741
 0.0017253478963870853
 0.0016885275841076281
 0.0016524313261302747
 0.0016170462998192799
 0.0015823598788515954

In [21]:
dist = Hypergeometric(10, 10, 10)
a = pvalue_stern.(dist, -1:11)
b = pvalue_stern_naive.(dist, -1:11)
@show a ≈ b
a

a ≈ b = true


13-element Vector{Float64}:
 0.0
 1.082508822446903e-5
 0.0010933339106713729
 0.02301413756522111
 0.1788954079975756
 0.6562817986966591
 1.0
 0.6562817986966591
 0.1788954079975756
 0.02301413756522111
 0.0010933339106713729
 1.082508822446903e-5
 0.0

In [22]:
dist = Poisson(4)
a = pvalue_stern.(dist, -1:10)
b = pvalue_stern_naive.(dist, -1:10)
@show a ≈ b
a

a ≈ b = true


12-element Vector{Float64}:
 0.0
 0.03967907337671833
 0.20225217284624464
 0.45297291852313915
 1.0
 1.0
 0.6092663703736709
 0.3064478074132657
 0.1289896172913079
 0.06944925468158153
 0.02136343448798415
 0.008132242796933864